In [ ]:
#### ROUGH --- FROM DEV #####

In [ ]:
def get_query_head_facts(pw_obj, idx=None):
    heads = {h: {} for h in set([t[0] for t in pw_obj.rls['ruleH_1']])}
    true_heads = set([t[0] for t in pw_obj.rls['ruleHTrue_1']])
    for row in pw_obj.rls['newHArc_3']:
        pos, h, var = int(row[1]), row[2], row[0]
        heads[h][pos] = var
    head_strs = []
    for h in sorted(heads.keys()):
        variables = [(heads[h][k]) for k in sorted(heads[h].keys())]
        is_true = h in true_heads
        h_str = '{}{}{}({}).'.format('' if is_true else 'n', head_to_node(h), '' if idx is None else str(idx), ','.join(variables))
        head_strs.append(h_str)
    return head_strs

In [ ]:
def get_equivalent_sets(objs, match_func):
    
    sets = []
    curr_iter_set = list(range(len(objs)))
    while len(curr_iter_set) > 0:
        next_iter_set = []
        seed = curr_iter_set[0]
        curr_set = {objs[seed]}
        for c in curr_iter_set[1:]:
            if match_func(objs[seed], objs[c]):
                curr_set.add(objs[c])
            else:
                next_iter_set.append(c)
        sets.append(curr_set)
        curr_iter_set = next_iter_set
    return sets

In [ ]:
from IPython.display import HTML as html_print

def cstr(s, color='black'):
    return "<text style=color:{}>{}</text>".format(color, s)

In [ ]:
def rule_to_node(atom, occ):
    return "{}[{}]".format(atom.strip('"'), occ)

def head_to_node(head):
    return remove_double_quotes(head)

def remove_double_quotes(r):
    return r.strip('"')

def print_colored_query_string(pw_rel_dfs):
    
    heads = {h: {} for h in set(pw_rel_dfs['ruleH_1']['HEAD'])}
    rules = {}
    for _, p in pw_rel_dfs['ruleOcc_2'][['ATOM', 'OCC']].iterrows():
        rules[(p['ATOM'], int(p['OCC']))] = {}
    
    true_heads = set(pw_rel_dfs['ruleHTrue_1']['HEAD'])
    true_rules = set([])
    for _, p in pw_rel_dfs['ruleOccTrue_2'][['ATOM', 'OCC']].iterrows():
        true_rules.add((p['ATOM'], int(p['OCC'])))
    
    for _, row in pw_rel_dfs['newHArc_3'].iterrows():
        pos = int(row['POS'])
        h = row['HEAD']
        var = row['VAR']
        heads[h][pos] = var
    for _, row in pw_rel_dfs['newArc_4'].iterrows():
        rule = (row['ATOM'], int(row['OCC']))
        pos = int(row['POS'])
        var = row['VAR']
        rules[rule][pos] = var
        
#     print(heads, rules, true_heads, true_rules)
    
    heads_htmls = []
    for h in sorted(heads.keys()):
        variables = [remove_double_quotes(heads[h][k]) for k in sorted(heads[h].keys())]
        heads_htmls.append(cstr('{}({})'.format(head_to_node(h), ','.join(variables)), 'green' if h in true_heads else 'red'))
    
    
    rules_htmls = []
    for (r, occ) in sorted(rules.keys()):
        variables = [remove_double_quotes(rules[(r,occ)][k]) for k in sorted(rules[(r,occ)].keys())]
        rules_htmls.append(cstr('{}({})'.format(head_to_node(r), ','.join(variables)), 'green' if (r,occ) in true_rules else 'red'))
    
#     display(html_print(cstr(' '.join(heads_htmls + rules_htmls))))
    
    
    heads_htmls_combined = cstr(' ; '.join(heads_htmls))
    rules_htmls_combined = cstr(', '.join(rules_htmls))
    display(html_print(cstr('{} :- {}.'.format(heads_htmls_combined, rules_htmls_combined))))
    
    eqls = []
    for _, row in pw_rel_dfs['eqOrd_3'].iterrows():
        eqls.append('{} = {}'.format(row['VAR1'], row['VAR2']))
    print('where' if len(eqls) > 0 else '')
    print(' and '.join(eqls))
    
    
def get_pattern_graph(pw_rel_dfs):
    g = nx.MultiDiGraph()
    true_heads = set(pw_rel_dfs['ruleHTrue_1']['HEAD'])
    if len(true_heads) > 0:
        print("Success Pattern:")
    else:
        print("Failure Pattern:")
    
    if 'e_3' in pw_rel_dfs:
        for _, row in pw_rel_dfs['e_3'].iterrows():
            g.add_edge(row['NODE1'], row['NODE2'], color='darkgreen', label='{}{}'.format('e', row['OCC']))
    if 'ne_3' in pw_rel_dfs:
        for _, row in pw_rel_dfs['ne_3'].iterrows():
            g.add_edge(row['NODE1'], row['NODE2'], color='red', style='dotted', label='{}{}'.format('e', row['OCC']))
    
    return g

def get_pattern_graph2(pw_rel_dfs, no_node_labels=False):
    g = nx.DiGraph()
    true_heads = set(pw_rel_dfs['ruleHTrue_1']['HEAD'])
    if len(true_heads) > 0:
        print("Success Pattern:")
    else:
        print("Failure Pattern:")
    
    if 'e_3' in pw_rel_dfs:
        for _, row in pw_rel_dfs['e_3'].iterrows():
            if (row['NODE1'], row['NODE2']) in g.edges:
                g.edges[(row['NODE1'], row['NODE2'])]['label'] += ';{}{}'.format('e', row['OCC'])
            else:
                g.add_edge(row['NODE1'], row['NODE2'], color='darkgreen', label='{}{}'.format('e', row['OCC']))
    if 'ne_3' in pw_rel_dfs:
        for _, row in pw_rel_dfs['ne_3'].iterrows():
            if (row['NODE1'], row['NODE2']) in g.edges:
                g.edges[(row['NODE1'], row['NODE2'])]['label'] += ';{}{}'.format('e', row['OCC'])
            else:
                g.add_edge(row['NODE1'], row['NODE2'], color='red', style='dotted', label='{}{}'.format('e', row['OCC']))
    
    if no_node_labels:
        for n in g.nodes:
            g.nodes[n]['label'] = '  '
    
    return g

def get_incidence_graph(pw_rel_dfs):
    
    g = nx.MultiDiGraph()
    g.graph['rankdir'] = 'LR'
    
    for _, row in pw_rel_dfs['ruleH_1'].iterrows():
        g.add_node(head_to_node(row['HEAD']), rank='min', color='red', shape='box', style='filled')
    for _, row in pw_rel_dfs['ruleHTrue_1'].iterrows():
        g.add_node(head_to_node(row['HEAD']), rank='min', color='green', style='filled')
    for _, row in pw_rel_dfs['ruleOcc_2'].iterrows():
        g.add_node(rule_to_node(row['ATOM'], row['OCC']), color='red', style='filled')
    for _, row in pw_rel_dfs['ruleOccTrue_2'].iterrows():
        g.add_node(rule_to_node(row['ATOM'], row['OCC']), color='green', style='filled')
    
    
    for _, row in pw_rel_dfs['newHArc_3'].iterrows():
        g.add_edge(row['VAR'], head_to_node(row['HEAD']), label=row['POS'])
    for _, row in pw_rel_dfs['newArc_4'].iterrows():
        g.add_edge(row['VAR'], rule_to_node(row['ATOM'], row['OCC']), label=row['POS'])
    
    for _, row in pw_rel_dfs['eqOrd_3'].iterrows():
        g.add_edge(row['VAR2'], row['VAR1'], constraint='false', color='green', style='dotted')
    
    return g

In [ ]:
def eq_groups(pw_rel_dfs, single_grps=False):
    
    p = pw_rel_dfs['eqOrdMinimal_3'].groupby(['VAR1'])
    new_vars = set(pw_rel_dfs['newVar_2']['VAR'])
    groups = []
    for var1, group in p.groups.items():
        new_vars.remove(var1)
        groups.append([var1]+[pw_rel_dfs['eqOrdMinimal_3'].loc[i]['VAR2'] for i in group])
    if single_grps:
        for v in new_vars:
            groups.append([v])
    return groups
    

def chain_eqs_rules(pw_rel_dfs):
    
    eqls = []
    groups = eq_groups(pw_rel_dfs)
    for group in groups:
        eqls.append('='.join(map(remove_double_quotes, group)))
    
    return eqls

def eq_rules(pw_rel_dfs):
    
    eqls = []
    groups = eq_groups(pw_rel_dfs)
    
    for group in groups:
        var1 = group[0]
        for var2 in group[1:]:
            eqls.append('{}={}'.format(remove_double_quotes(var1), remove_double_quotes(var2)))
    
    return eqls

def neq_rules(pw_rel_dfs):
    neqls = []
    for _, row in pw_rel_dfs['neqOrd_3'].iterrows():
        neq_str = '{} != {}'.format(remove_double_quotes(row['VAR1']), remove_double_quotes(row['VAR2']))
        neqls.append(cstr(neq_str))
    return neqls

def get_query_heads(ruleH_dfs, ruleHTrue_dfs, hArc_df, colored=True):
    
    heads = {h: {} for h in set(ruleH_dfs['HEAD'])}
    true_heads = set(ruleHTrue_dfs['HEAD'])
    for _, row in hArc_df.iterrows():
        pos = int(row['POS'])
        h = row['HEAD']
        var = row['VAR']
        heads[h][pos] = var
    heads_htmls = []
    for h in sorted(heads.keys()):
        variables = [remove_double_quotes(heads[h][k]) for k in sorted(heads[h].keys())]
        is_true = h in true_heads
        h_str = '{}{}({})'.format('' if is_true else 'n', head_to_node(h), ','.join(variables))
        if colored:
            heads_htmls.append(cstr(h_str, 'green' if is_true else 'red'))
        else:
            heads_htmls.append(cstr(h_str))
    return heads_htmls

def get_original_query_heads(pw_rel_dfs, colored=True):
    
    return get_query_heads(pw_rel_dfs['ruleH_1'], pw_rel_dfs['ruleHTrue_1'], pw_rel_dfs['hArc_3'], colored)

def get_substituted_query_heads(pw_rel_dfs, colored=True):
    
    return get_query_heads(pw_rel_dfs['ruleH_1'], pw_rel_dfs['ruleHTrue_1'], pw_rel_dfs['newHArc_3'], colored)

def get_query_body_rules(ruleOcc_df, ruleOccTrue_df, arc_df, colored=True):
    
    rules = {}
    for _, p in ruleOcc_df[['ATOM', 'OCC']].iterrows():
        rules[(p['ATOM'], int(p['OCC']))] = {}
    
    true_rules = set([])
    for _, p in ruleOccTrue_df[['ATOM', 'OCC']].iterrows():
        true_rules.add((p['ATOM'], int(p['OCC'])))
    
    for _, row in arc_df.iterrows():
        rule = (row['ATOM'], int(row['OCC']))
        pos = int(row['POS'])
        var = row['VAR']
        rules[rule][pos] = var
    
    rules_htmls = []
    for (r, occ) in sorted(rules.keys()):
        variables = [remove_double_quotes(rules[(r,occ)][k]) for k in sorted(rules[(r,occ)].keys())]
        is_true = (r,occ) in true_rules
        r_str = '{}{}({})'.format('' if is_true else 'not ', head_to_node(r), ','.join(variables))
        if colored:
            rules_htmls.append(cstr(r_str, 'green' if is_true else 'red'))
        else:
            rules_htmls.append(cstr(r_str))
    
    return rules_htmls

def get_original_query_body_rules(pw_rel_dfs, colored=True):
    
    return get_query_body_rules(pw_rel_dfs['ruleOcc_2'], pw_rel_dfs['ruleOccTrue_2'], pw_rel_dfs['arc_4'], colored)

def get_substituted_query_body_rules(pw_rel_dfs, colored=True):
    
    return get_query_body_rules(pw_rel_dfs['ruleOcc_2'], pw_rel_dfs['ruleOccTrue_2'], pw_rel_dfs['newArc_4'], colored)
    
def print_rewritten_query_string(pw_rel_dfs):

    heads_htmls = get_substituted_query_heads(pw_rel_dfs, colored=True)
    rules_htmls = get_substituted_query_body_rules(pw_rel_dfs, colored=True)
    neqls = neq_rules(pw_rel_dfs)
    rules_htmls.extend(neqls)
    
    heads_htmls_combined = ' ; '.join(heads_htmls)
    rules_htmls_combined = ', '.join(rules_htmls)
    
    display(html_print(cstr('{} :- {}.'.format(heads_htmls_combined, rules_htmls_combined))))
    

def print_explicit_rewritten_query_string(pw_rel_dfs, chain_eq=True):
    
    heads_htmls = get_original_query_heads(pw_rel_dfs, colored=True)
    rules_htmls = get_original_query_body_rules(pw_rel_dfs, colored=True)
    eqls = chain_eqs_rules(pw_rel_dfs) if chain_eq else eq_rules(pw_rel_dfs)
    neqls = neq_rules(pw_rel_dfs)
    rules_htmls.extend(eqls)
    rules_htmls.extend(neqls)
    
    heads_htmls_combined = ' ; '.join(heads_htmls)
    rules_htmls_combined = ', '.join(rules_htmls)
    
    display(html_print(cstr('{} :- {}.'.format(heads_htmls_combined, rules_htmls_combined))))
    
def print_fancy_rewrite(pw_rel_dfs):
    
    heads_htmls = get_original_query_heads(pw_rel_dfs, colored=True)
    rules_htmls = get_original_query_body_rules(pw_rel_dfs, colored=True)
    eq_grps = eq_groups(pw_rel_dfs, single_grps=True)
    eqls = ['='.join(map(remove_double_quotes, grp)) for grp in eq_grps]
    eqls = ['[{}]'.format(eq_grp) for eq_grp in eqls]
    
    heads_htmls_combined = ' ; '.join(heads_htmls)
    rules_htmls_combined = ', '.join(rules_htmls)
    eqls_combined = cstr(''.join(eqls), 'blue')
    
    display(html_print(cstr('{} :- {}. % {}'.format(heads_htmls_combined, rules_htmls_combined, eqls_combined))))

In [ ]:
def match(a,b):
    if (a==1 and b==3) or (a==1 and b==5) or (a==2 and b==4):
        return True
    return False

In [ ]:
sets = []

curr_iter_list = [1,2,3,4,5,6]
while len(curr_iter_list) > 0:
    print("curr_iter_list:", curr_iter_list)
    next_iter_list = []
    seed = curr_iter_list[0]
    print("seed:", seed)
    curr_set = {seed}
    for c in curr_iter_list[1:]:
        if match(seed, c):
            curr_set.add(c)
            print("Match", seed, c)
        else:
            next_iter_list.append(c)
    sets.append(curr_set)
    curr_iter_list = next_iter_list

sets

// Helpful: https://stackoverflow.com/questions/44274518/how-can-i-control-within-level-node-order-in-graphvizs-dot

digraph test{

rankdir = LR;
label = PW12;

// make invisible ranks
rankRuleNode [style=invisible];

rankRedundantVars [style=invis];
rankVars [style=invis];

TRI [shape=box color=red style=filled];
E1 [color=green style=filled];
E2 [color=red style=filled];
E3 [color=red style=filled];

rankRedundantVars  -> rankVars -> rankRuleNode [style=invis];

// declare nodes all out of desired order

X -> E3 [constraint = false label=1];
X -> E3 [constraint = false label=2];
X -> E1 [constraint = false label=1];
Y -> E1 [constraint = false label=2];
Y -> E2 [constraint = false label=1];
Y -> E2 [constraint = false label=2];
X -> TRI [constraint = false label=1];
X -> TRI [constraint = false label=3];
Y -> TRI [constraint = false label=2];

Z -> X [constraint=true style=dotted color=green];

{
rank = same;
// Here you enforce the desired order with "invisible" edges and arrowheads
rankVars -> X -> Y [ style=invis ];
rankdir = TB;
}

{
rank = same;
// Here you enforce the desired order with "invisible" edges and arrowheads
rankRedundantVars -> Z [ style=invis ];
rankdir = TB;
}


{
rank = same;
// Here you enforce the desired order with "invisible" edges and arrowheads
rankRuleNode -> TRI -> E1 -> E2 -> E3 [ style=invis ];
rankdir = TB;
}


}

In [ ]:
html_print(cstr(' '.join([cstr('HELLO', 'blue'), cstr('WORLD', 'red')])))

In [ ]:
%%clingo --donot-display_input -exp query_analysis

% schema var(VAR, DOMAIN).
var("X", node).
var("Y", node).
var("Z", node).

% schema ruleOcc(ATOM_OCC).
ruleOcc("e1"). % ruleOcc("e1", 2) instead? 2 == arity
ruleOcc("e2").
ruleOcc("e3").
ruleOcc("tri").

% schema ruleOccTrue(ATOM_OCC)
ruleOccTrue(ROC) ; not ruleOccTrue(ROC) :- ruleOcc(ROC), ROC != "tri".
ruleOccTrue("tri") :- ruleOccTrue("e1"), ruleOccTrue("e2"), ruleOccTrue("e3").

% schema eqether(VAR1, VAR2, DOMAIN) --> eq()
% should eq(A,A) be a thing?
eq(A,B,TYPE) ; not eq(A,B,TYPE) :- var(A, TYPE), var(B, TYPE), A!=B.
eq(A,B,TYPE) :- eq(B,A,TYPE).
eq(A,B,TYPE) :- eq(A,C,TYPE), eq(C,B,TYPE), A!=B, B!=C, A!=C.

eqOrd(A,B,TYPE) :- eq(A,B,TYPE), A<B.

% schema arc(VAR, POSITION, ATOM_OCC)
arc("X", 1, "e1").
arc("Y", 2, "e1").
arc("Y", 1, "e2").
arc("Z", 2, "e2").
arc("Z", 1, "e3").
arc("X", 2, "e3").
arc("X", 1, "tri").
arc("Y", 2, "tri").
arc("Z", 3, "tri").

% Find the new arcs, getting rid of the redundancies
newArc(VAR, POS, ATOM_OCC) :- arc(VAR_ORIG, POS, ATOM_OCC), eqOrd(VAR,VAR_ORIG,TYPE), not eqOrd(_,VAR,TYPE).
newArc(VAR, POS, ATOM_OCC) :- arc(VAR, POS, ATOM_OCC), not eqOrd(_,VAR,_).


% It cannot be true that e1 and e2 have the same variable assignments but one is True while other is False
:- newArc(V1, 1, "e1"), newArc(V2, 2, "e1"), newArc(V1, 1, "e2"), newArc(V2, 2, "e2"), ruleOccTrue("e1"), not ruleOccTrue("e2").
:- newArc(V1, 1, "e1"), newArc(V2, 2, "e1"), newArc(V1, 1, "e2"), newArc(V2, 2, "e2"), ruleOccTrue("e2"), not ruleOccTrue("e1").

:- newArc(V1, 1, "e2"), newArc(V2, 2, "e2"), newArc(V1, 1, "e3"), newArc(V2, 2, "e3"), ruleOccTrue("e2"), not ruleOccTrue("e3").
:- newArc(V1, 1, "e2"), newArc(V2, 2, "e2"), newArc(V1, 1, "e3"), newArc(V2, 2, "e3"), ruleOccTrue("e3"), not ruleOccTrue("e2").

:- newArc(V1, 1, "e1"), newArc(V2, 2, "e1"), newArc(V1, 1, "e3"), newArc(V2, 2, "e3"), ruleOccTrue("e1"), not ruleOccTrue("e3").
:- newArc(V1, 1, "e1"), newArc(V2, 2, "e1"), newArc(V1, 1, "e3"), newArc(V2, 2, "e3"), ruleOccTrue("e3"), not ruleOccTrue("e1").

% New inferred edges
e(V1,V2) :- newArc(V1,1,"e1"), newArc(V2,2,"e1"), ruleOccTrue("e1").
e(V1,V2) :- newArc(V1,1,"e2"), newArc(V2,2,"e2"), ruleOccTrue("e2").
e(V1,V2) :- newArc(V1,1,"e3"), newArc(V2,2,"e3"), ruleOccTrue("e3").

ne(V1,V2) :- newArc(V1,1,"e1"), newArc(V2,2,"e1"), not ruleOccTrue("e1").
ne(V1,V2) :- newArc(V1,1,"e2"), newArc(V2,2,"e2"), not ruleOccTrue("e2").
ne(V1,V2) :- newArc(V1,1,"e3"), newArc(V2,2,"e3"), not ruleOccTrue("e3").

% Given these inferred edges, can a triangle exist?
tri(V1,V2,V3) :- e(V1,V2), e(V2,V3), e(V3,V1).
triExists :- tri(_,_,_).

%graphviz graph graph_type=directed rankdir=LR
%graphviz edge newArc(HEAD, _, TAIL) label=$2
%graphviz node ruleOccTrue(N) color=green

% :- not ruleOccTrue("tri").

% :- triExists, not ruleOccTrue("tri"). % --> The exact assignment doesn't work, but some alignment does.


#show ruleOccTrue/1.
% #show eqOrd/3.
#show newArc/3.
#show e/2.
#show ne/2.
#show triExists/0.
#show tri/3.






In [ ]:
temp_df = query_analysis['pw_rel_dfs']['eqOrdMinimal_3']
temp_df = temp_df[temp_df['pw'] == 15]
temp_df

In [ ]:
p = temp_df.groupby(['VAR1'])

In [ ]:
p.groups

In [ ]:
for k, v in p.groups.items():
    for i in v:
        print(temp_df.loc[i]['VAR2'])

In [ ]:
# query_analysis['pw_rel_dfs'], query_analysis['rel_schema'], query_analysis['pw_objs'] = load_worlds(query_analysis['asp_soln'],
#                                                                                         query_analysis['meta_data'])




In [ ]:
# for i in range(1, len(query_analysis['pw_objs'])+1):
#     print("PW:", i)
#     pw_rel_dfs = pw_slicer(query_analysis['pw_rel_dfs'], None, [i])[0]
    
    
#     g = nx.MultiDiGraph()
#     g.graph['rankdir'] = 'LR'
    
#     g.add_node('"tri"')
    
#     for _, row in pw_rel_dfs['newArc_3'].iterrows():
#         g.add_edge(row['x1'], row['x3'], label=row['x2'])
#     for n in ['"e1"', '"e2"', '"e3"', '"tri"']:
#         g.nodes[n]['color'] = 'red'
#     for _, row in pw_rel_dfs['ruleOccTrue_1'].iterrows():
#         g.nodes[row['ATOM_OCC']]['color'] = 'green'
        
#     display(draw(g))
    
#     g = nx.MultiDiGraph()
    
    
#     if '"tri"' in list(pw_rel_dfs['ruleOccTrue_1']['ATOM_OCC']):
#         print("Success Pattern:")
#     else:
#         print("Failure Pattern:")
        
#     if len(pw_rel_dfs['triExists_0']) > 0:
#         print("Triangle Exists")
    
#     for _, row in pw_rel_dfs['e_2'].iterrows():
#         g.add_edge(row['x1'], row['x2'])
# #     for _, row in pw_rel_dfs['ne_2'].iterrows():
# #         g.add_edge(row['x1'], row['x2'], color='red', style='dotted')
    
    
#     display(draw(g))

In [ ]:
%%clingo --donot-display_input --donot-run

% schema var(VAR, DOMAIN).
var("X", node).
var("Y", node).
var("Z", node).

tog(A,B,TYPE) ; not tog(A,B,TYPE) :- var(A, TYPE), var(B, TYPE), A!=B.
tog(A,B,TYPE) :- tog(B,A,TYPE).
tog(A,B,TYPE) :- tog(A,C,TYPE), tog(C,B,TYPE), A!=B, B!=C, A!=C.

togOrd(A,B,TYPE) :- tog(A,B,TYPE), A<B.

newVar(A, TYPE) :- var(A, TYPE), not togOrd(_,A,TYPE).


#show newVar/2.
#show togOrd/3.



In [ ]:
%%clingo --donot-display_input -exp query_analysis

% schema var(VAR, DOMAIN).
var("X", node).
var("Y", node).
var("Z", node).

% schema ruleOcc(ATOM, OCC).
ruleOcc("e", 1..3).

% schema ruleH(HEAD)
ruleH("tri").

% schema ruleOccTrue(ATOM, OCC)
ruleOccTrue(R, OC) ; not ruleOccTrue(R, OC) :- ruleOcc(R, OC).

% schema ruleHTrue(HEAD)
ruleHFalse(H) :- ruleH(H), ruleOcc(R, OCC), not ruleOccTrue(R,OCC).
ruleHTrue(H)  :- ruleH(H), not ruleHFalse(H).

% schema eq(VAR1, VAR2, DOMAIN)
eq(A,B,TYPE) ; not eq(A,B,TYPE) :- var(A, TYPE), var(B, TYPE), A!=B.
eq(A,B,TYPE) :- eq(B,A,TYPE).
eq(A,B,TYPE) :- eq(A,C,TYPE), eq(C,B,TYPE), A!=B, B!=C, A!=C.

% schema eqOrd(VAR1, VAR2, DOMAIN)
eqOrd(A,B,TYPE) :- eq(A,B,TYPE), A<B.

% schema arc(VAR, POS, ATOM, OCC)
arc("X", 1, "e", 1).
arc("Y", 2, "e", 1).
arc("Y", 1, "e", 2).
arc("Z", 2, "e", 2).
arc("Z", 1, "e", 3).
arc("X", 2, "e", 3).

% schema hArc(VAR, POS, HEAD)
hArc("X", 1, "tri") :- ruleH("tri").
hArc("Y", 2, "tri") :- ruleH("tri").
hArc("Z", 3, "tri") :- ruleH("tri").

% Find the new arcs, getting rid of the redundancies

% schema newArc(VAR, POS, ATOM, OCC)
newArc(VAR, POS, ATOM, OCC) :- arc(VAR_ORIG, POS, ATOM, OCC), eqOrd(VAR,VAR_ORIG,TYPE), not eqOrd(_,VAR,TYPE).
newArc(VAR, POS, ATOM, OCC) :- arc(VAR, POS, ATOM, OCC), not eqOrd(_,VAR,_).

% schema newHArc(VAR, POS, HEAD)
newHArc(VAR, POS, H) :- hArc(VAR_ORIG,POS,H), eqOrd(VAR,VAR_ORIG,TYPE), not eqOrd(_,VAR,TYPE).
newHArc(VAR, POS, H) :- hArc(VAR, POS, H), not eqOrd(_,VAR,_).

% It cannot be true that eX and eY have the same variable assignments but one is True while other is False
:- newArc(V1, 1, "e", OCC1), newArc(V2, 2, "e", OCC1), newArc(V1, 1, "e", OCC2), newArc(V2, 2, "e", OCC2), ruleOccTrue("e", OCC1), not ruleOccTrue("e", OCC2). 

eqAtomOccOrd("e", OCC1, OCC2) :- newArc(V1, 1, "e", OCC1), newArc(V2, 2, "e", OCC1), newArc(V1, 1, "e", OCC2), newArc(V2, 2, "e", OCC2), OCC1<OCC2.
eqAtomOccOrd("e", OCC1, OCC3) :- eqAtomOccOrd("e", OCC1, OCC2), eqAtomOccOrd("e", OCC2, OCC3).

fAtom("e", V1, V2) :- newArc(V1, 1, "e", OCC1), newArc(V2, 2, "e", OCC1), not eqAtomOccOrd("e",_,OCC1).


% New inferred edges
% schema e(NODE1, NODE2)
e(V1,V2) :- newArc(V1, 1, "e", OCC), newArc(V2, 2, "e", OCC), ruleOccTrue("e", OCC).

% New inferred missing edges
% schema ne(NODE1, NODE2)
ne(V1,V2) :- newArc(V1, 1, "e", OCC), newArc(V2, 2, "e", OCC), not ruleOccTrue("e", OCC).


% Given these inferred edges, can a triangle exist?
% tri(V1,V2,V3) :- e(V1,V2), e(V2,V3), e(V3,V1).
% triExists :- tri(_,_,_).

%graphviz graph graph_type=directed rankdir=LR
%graphviz edge newArc(HEAD, _, TAIL) label=$2
%graphviz node ruleOccTrue(N) color=green

% :- not ruleOccTrue("tri").


#show ruleOccTrue/2.
#show var/2.
#show ruleOcc/2.
#show ruleH/1.
#show eqOrd/3.
#show newArc/4.
#show newHArc/3.
#show e/2.
#show ne/2.We 
#show eqAtomOccOrd/3.
#show fAtom/3.
% #show triExists/0.
% #show tri/3.
#show ruleHTrue/1.








In [ ]:
%%clingo --donot-display_input -exp query_analysis

% schema var(VAR, DOMAIN).
var("X", node).
var("Y", node).
var("Z", node).

% schema eq(VAR1, VAR2, DOMAIN)
eq(A,A,TYPE) :- var(A,TYPE).
eq(A,B,TYPE) ; not eq(A,B,TYPE) :- var(A, TYPE), var(B, TYPE).
eq(A,B,TYPE) :- eq(B,A,TYPE).
eq(A,B,TYPE) :- eq(A,C,TYPE), eq(C,B,TYPE).

% schema eqOrd(VAR1, VAR2, DOMAIN)
eqOrd(A,B,TYPE) :- eq(A,B,TYPE), A<=B.

% schema ruleOcc(ATOM, OCC).
ruleOcc("e", 1..3).

% schema ruleH(HEAD)
ruleH("tri").

% ruleOccEq(R, OCC, OCC) :- ruleOcc(R, OCC).
% ruleOccEq(R, OCC1, OCC2) ; not ruleOccEq(R, OCC1, OCC2) :- ruleOcc(R, OCC1), ruleOcc(R, OCC2).
% ruleOccEq(R, OCC1, OCC2) :- ruleOccEq(R, OCC2, OCC1).
% ruleOccEq(R, OCC1, OCC2) :- ruleOccEq(R, OCC1, OCC3), ruleOccEq(R, OCC3, OCC2).

% ruleOccEqOrd(R, OCC1, OCC2) :- ruleOccEq(R, OCC1, OCC2), OCC1 < OCC2.

% schema arc(VAR, POS, ATOM, OCC)
arc("X", 1, "e", 1).
arc("Y", 2, "e", 1).
arc("Y", 1, "e", 2).
arc("Z", 2, "e", 2).
arc("Z", 1, "e", 3).
arc("X", 2, "e", 3).

e(V1,V2) :- arc(V1, 1, "e", OCC), arc(V2, 2, "e", OCC).
e(V1,V2) :- e(V3,V4), eq(V1,V3,TYPE1), eq(V2,V4,TYPE2).

% arc(V1, 1, "e", OCC) :- e(V1, _), ruleOcc("e", OCC). 
% arc(V2, 2, "e", OCC) :- e(_, V2), ruleOcc("e", OCC). 

% schema hArc(VAR, POS, HEAD)
hArc("X", 1, "tri") :- ruleH("tri").
hArc("Y", 2, "tri") :- ruleH("tri").
hArc("Z", 3, "tri") :- ruleH("tri").

#show eqOrd/3.
#show e/2.
% #show arc/4.





In [ ]:
#     heads = {h: {} for h in set(pw_rel_dfs['ruleH_1']['HEAD'])}
#     rules = {}
#     for _, p in pw_rel_dfs['ruleOcc_2'][['ATOM', 'OCC']].iterrows():
#         rules[(p['ATOM'], int(p['OCC']))] = {}
    
#     true_heads = set(pw_rel_dfs['ruleHTrue_1']['HEAD'])
#     true_rules = set([])
#     for _, p in pw_rel_dfs['ruleOccTrue_2'][['ATOM', 'OCC']].iterrows():
#         true_rules.add((p['ATOM'], int(p['OCC'])))
    
#     for _, row in pw_rel_dfs['hArc_3'].iterrows():
#         pos = int(row['POS'])
#         h = row['HEAD']
#         var = row['VAR']
#         heads[h][pos] = var
#     for _, row in pw_rel_dfs['arc_4'].iterrows():
#         rule = (row['ATOM'], int(row['OCC']))
#         pos = int(row['POS'])
#         var = row['VAR']
#         rules[rule][pos] = var
        
#     # print(heads, rules, true_heads, true_rules)
    
#     heads_htmls = []
#     for h in sorted(heads.keys()):
#         variables = [heads[h][k] for k in sorted(heads[h].keys())]
#         is_true = h in true_heads
#         h_str = '{}{}({})'.format('' if is_true else 'n', head_to_node(h), ','.join(variables))
#         heads_htmls.append(cstr(h_str, 'green' if is_true else 'red'))
    
    
#     rules_htmls = []
#     for (r, occ) in sorted(rules.keys()):
#         variables = [rules[(r,occ)][k] for k in sorted(rules[(r,occ)].keys())]
#         is_true = (r,occ) in true_rules
#         r_str = '{}{}({})'.format('' if is_true else 'not ', head_to_node(r), ','.join(variables))
#         rules_htmls.append(cstr(r_str, 'green' if is_true else 'red'))
    
    # display(html_print(cstr(' '.join(heads_htmls + rules_htmls))))

In [ ]:
#     heads = {h: {} for h in set(pw_rel_dfs['ruleH_1']['HEAD'])}
#     rules = {}
#     for _, p in pw_rel_dfs['ruleOcc_2'][['ATOM', 'OCC']].iterrows():
#         rules[(p['ATOM'], int(p['OCC']))] = {}
    
#     true_heads = set(pw_rel_dfs['ruleHTrue_1']['HEAD'])
#     true_rules = set([])
#     for _, p in pw_rel_dfs['ruleOccTrue_2'][['ATOM', 'OCC']].iterrows():
#         true_rules.add((p['ATOM'], int(p['OCC'])))
    
#     for _, row in pw_rel_dfs['newHArc_3'].iterrows():
#         pos = int(row['POS'])
#         h = row['HEAD']
#         var = row['VAR']
#         heads[h][pos] = var
#     for _, row in pw_rel_dfs['newArc_4'].iterrows():
#         rule = (row['ATOM'], int(row['OCC']))
#         pos = int(row['POS'])
#         var = row['VAR']
#         rules[rule][pos] = var
        
#     # print(heads, rules, true_heads, true_rules)
    
#     heads_htmls = []
#     for h in sorted(heads.keys()):
#         variables = [heads[h][k] for k in sorted(heads[h].keys())]
#         is_true = h in true_heads
#         h_str = '{}{}({})'.format('' if is_true else 'n', head_to_node(h), ','.join(variables))
#         heads_htmls.append(cstr(h_str, 'green' if is_true else 'red'))
    
    
#     rules_htmls = []
#     for (r, occ) in sorted(rules.keys()):
#         variables = [rules[(r,occ)][k] for k in sorted(rules[(r,occ)].keys())]
#         is_true = (r,occ) in true_rules
#         r_str = '{}{}({})'.format('' if is_true else 'not ', head_to_node(r), ','.join(variables))
#         rules_htmls.append(cstr(r_str, 'green' if is_true else 'red'))

In [ ]:
for pw_id in range(1, len(query_analysis['pw_objs'])+1):
    pw_rel_dfs = pw_slicer(query_analysis['pw_rel_dfs'], None, [pw_id])[0]
    print(get_query_head_facts(pw_rel_dfs, idx=2))

In [ ]:
def get_query_head_facts(pw_rel_dfs, idx=None):
    
    ruleH_dfs, ruleHTrue_dfs, hArc_df = pw_rel_dfs['ruleH_1'], pw_rel_dfs['ruleHTrue_1'], pw_rel_dfs['newHArc_3']
    heads = {h: {} for h in set(ruleH_dfs['HEAD'])}
    true_heads = set(ruleHTrue_dfs['HEAD'])
    for _, row in hArc_df.iterrows():
        pos, h, var = int(row['POS']), row['HEAD'], row['VAR']
        heads[h][pos] = var
    head_strs = []
    for h in sorted(heads.keys()):
        variables = [remove_double_quotes(heads[h][k]) for k in sorted(heads[h].keys())]
        is_true = h in true_heads
        h_str = '{}{}{}({}).'.format('' if is_true else 'n', head_to_node(h), '' if idx is None else str(idx), ','.join(variables))
        head_strs.append(h_str)
    return head_strs

In [ ]:
for pw_obj in query_analysis['pw_objs']:
    print(get_query_head_facts(pw_obj, idx=5))